# Testing Sign-Language-MNIST models on live webcam data

In [1]:
import numpy as np
import cv2
import os
import glob
import string

In [2]:
from tensorflow import keras

model_1 = keras.models.load_model("./Best_CNN.h5")
model_2 = keras.models.load_model("./Underfitted_CNN.h5")

models = [('25 ep', model_1), (' 6 ep', model_2)]

capture = cv2.VideoCapture(0)
letters = [el for el in string.ascii_lowercase]
save_num = 0

save_folder = './saves'
roi_top_left = (320, 100)
roi_bottom_right = (620, 400)
color = (0,255,0)
font_size = 1.2
font_thickness = 2
flip_input = False

# cleanup old saved screenshots
old_saves = glob.glob(save_folder + '/*')
for old_save in old_saves:
    os.remove(old_save)
    
def evaluate_prediction(model, pred_input, class_names):
    pred = model.predict(pred_input)
    pred_index = int(np.argmax(pred, axis=-1)[0])
    pred_class_name = class_names[pred_index]
    pred_confidence = pred[0][pred_index]
    
    return (pred, pred_index, pred_class_name, pred_confidence)

while True:
    try:
        ret, frame = capture.read()
        frame = cv2.flip(frame, 1)
        roi = frame[roi_top_left[1]:roi_bottom_right[1], roi_top_left[0]:roi_bottom_right[0]]

        # convert region of interest
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi = cv2.resize(roi, (28,28), interpolation = cv2.INTER_CUBIC)
        if not flip_input:
            roi = cv2.flip(roi, 1)

        cv2.rectangle(frame, roi_top_left, roi_bottom_right, (0,255,0), 3)

        prediction_input = roi.reshape(1,28,28,1)
        prediction_input = prediction_input.astype('float32') / 255
        
        if len(models) > 1:
            predictions = []

            for mdl in models:
                model_name = mdl[0]
                model = mdl[1]    
                i = models.index(mdl)
                prediction, prediction_index, prediction_class, prediction_confidence = evaluate_prediction(model, 
                                                                                                            prediction_input, 
                                                                                                            letters)
                predictions.append(prediction)
                prediction_confidence = str(round(prediction_confidence,2))

                cv2.putText(frame, 
                            model_name + ': ' + prediction_class + ' [' + prediction_confidence + ']', 
                            (10,i*50 + 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            font_size, 
                            color, 
                            font_thickness)
        else:
            model = models[0][1]
            prediction, prediction_index, prediction_class, prediction_confidence = evaluate_prediction(model, 
                                                                                                        prediction_input, 
                                                                                                        letters)
            prediction_confidence = str(round(prediction_confidence,2))
            cv2.putText(frame, 
                        prediction_class + ' [' + prediction_confidence + ']', 
                        (300,80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        font_size, color, 
                        font_thickness)
        
        cv2.imshow('Prediction', frame)

        filename = save_folder + '/predicted_' + prediction_class + '_' + str(save_num) + '.jpg'
        keypress = cv2.waitKey(1)

        if keypress == ord('q'):
            break
        elif keypress == ord('c'):
            cv2.imwrite(filename, roi)
            save_num = save_num + 1
    except Exception as e:
        logging.error(traceback.format_exc())
        break
        
capture.release()
cv2.destroyAllWindows()